# Retail Sales and Customer Shopping Trends

#### Before beginning we need to import the necessary libraries to read the two csv files.

In [19]:
# Importing the pandas library for data manipulation and analysis
import pandas as pd

# Import the os module to perform operating system related tasks
import os

# Importing the numpy library for numerical operations
import numpy as np

# Importing the matplotlib library for creating static, animated, and interactive visualizations
import matplotlib.pyplot as plt

# Importing the seaborn library for making statistical graphics, built on top of matplotlib
import seaborn as sns

# Import the sqlite3 module to work with SQLite databases
import sqlite3

# Import the time module to work with time-related functions
import time

# Import the express module from the plotly library for interactive visualizations
import plotly.express as px

### Data Loading

In [22]:
# # Reading the CSV files from the 'data' folder
retail_df = pd.read_csv("data/Retail_Sales.csv")
customer_df = pd.read_csv('data/Customer_Shopping_Trends.csv')

# import os
# import pandas as pd

# base_path = r"C:\Users\Code Lou\OneDrive\Desktop\Retail-Sales-and-Customer-Shopping-Trends\data"
# retail_df = pd.read_csv(os.path.join(base_path, 'Retail_Sales.csv'))
# customer_df = pd.read_csv(os.path.join(base_path, 'Customer_Shopping_Trends.csv'))


FileNotFoundError: [Errno 2] No such file or directory: 'data/Retail_Sales.csv'

### Inspecting The Data
### Exploring the data to understand its structure

In [ ]:
#Looking at the first two rows of both datasets
superstore_df.head(2)
#Looking at the first two rows of both datasets
retail_df.head(2)

In [ ]:
#Looking at the last two rows of both datasets
superstore_df.tail(2)
retail_df.tail(2)

In [ ]:
# Displaying basic information of both datasets
superstore_df.info()
retail_df.info()

# Data Cleaning 
### Checking for duplicates.

In [ ]:
# Using conditional statements
if superstore_df.duplicated().sum() > 0:
    print("Duplicates are present")
else:
    print("No duplicates exist")   

In [ ]:
# Counting number of duplicate rows
retail_df.duplicated().sum()  
# Counting total duplicate columns
duplicate_count = retail_df.duplicated().sum()
duplicate_count

#### Now let's drop unecessary columns in both datasets

In [ ]:
# Lists of columns to drop from superstore_data
columns_to_drop = ['Row ID','Order ID', 'Ship Date', 'Customer Name', 'Ship Mode','Country', 'Postal Code','Product ID','City', 'Product Name']
superstore_df.drop(columns= columns_to_drop, axis=1, inplace=True)
# Columns to drop from retail_df
columns_to_drop = ['Transaction ID']
retail_df.drop(columns= columns_to_drop, axis=1, inplace=True)

Adding columns to calculate "Price per Unit" for "Superstore Sales", "Discount" and "Profit" for "Retail Sales"

In [ ]:
# Calculating 'Price per Unit' for Superstore Sales
superstore_df['Price_per_Unit'] = superstore_df['Sales'] / superstore_df['Quantity']

# Assigning 'Price_per_Unit' to retail_df before calculating 'Discount' and 'Profit'
retail_df['Price_per_Unit'] = retail_df['Total Amount'] / retail_df['Quantity']

# Calculating 'Discount' for Retail Sales
retail_df['Retail_Discount'] = retail_df['Total Amount'] - (retail_df['Price_per_Unit'] * retail_df['Quantity'])

# Calculating 'Profit' for Retail Sales
retail_df['Profit'] = (retail_df['Price_per_Unit'] * retail_df['Quantity']) - retail_df['Retail_Discount']

In [ ]:
# Converting date columns to datetime format
retail_df['Date'] = pd.to_datetime(retail_df['Date'])
superstore_df['Order Date'] = pd.to_datetime(superstore_df['Order Date'])

Extracting useful features from the date columns, such as year, month, and day, to analyze Sales trends over time 

In [ ]:
# Extracting year, month, and day from date columns of the Superstore
superstore_df['Superstore_Year'] = superstore_df['Order Date'].dt.year
superstore_df['Superstore_Month'] = superstore_df['Order Date'].dt.month
superstore_df['Superstore_Day_Of_Week'] = superstore_df['Order Date'].dt.dayofweek

# Extract year, month, and day from date columns of the Retail
retail_df['Retail_Year'] = retail_df['Date'].dt.year
retail_df['Retail_Month'] = retail_df['Date'].dt.month
retail_df['Retail_Day_Of_Week'] = retail_df['Date'].dt.dayofweek

#### Renaming similar Columns to avoid conflicts when merging.

In [ ]:
#Renaming columns in Superstore dataset
superstore_df.rename(columns={
    'Customer ID': 'Superstore_Customer_ID',
    'Order Date': 'Superstore_Order_Date',
    'Category': 'Superstore_Category',
    'Sales': 'Superstore_Sales',
    'Quantity': 'Superstore_Quantity',
    'Category': 'Superstore_Product_Category',
    'Sub-Category': 'Superstore_Sub_Category',
    'Price_per_Unit': 'Superstore_Price_per_Unit'
},  inplace=True)

#Renaming columns in Retail dataset
retail_df.rename(columns={
    'Customer ID': 'Retail_Customer_ID',
    'Quantity': 'Retail_Quantity',
    'Date': 'Retail_Order_Date',
    'Category' : 'Retail_Category',
    'Total Amount' : 'Retail_Sales',
    'Product Category': 'Retail_Product_Category',
    'Price per Unit' : 'Retail_Price_per_Unit',
    'Profit': 'Retail_Profit'
}, inplace=True)

#### Reording columns to maintain consistency and readability when merging.

In [ ]:
# Reordering columns in Superstore dataset
superstore_df = superstore_df[['Superstore_Customer_ID', 'Superstore_Order_Date',
    'Segment', 'State', 'Region', 'Superstore_Product_Category',
    'Superstore_Sub_Category', 'Superstore_Quantity', 'Superstore_Price_per_Unit',
    'Superstore_Sales', 'Discount', 'Profit', 
    'Superstore_Year', 'Superstore_Month', 
    'Superstore_Day_Of_Week'
]]

# Reordering columns in Retail dataset
retail_df = retail_df[[ 'Retail_Customer_ID', 'Retail_Order_Date', 'Gender', 'Age', 
    'Retail_Product_Category', 'Retail_Quantity', 'Retail_Sales',
    'Retail_Discount' ,'Retail_Profit', 'Retail_Year', 'Retail_Month', 
    'Retail_Day_Of_Week'
]]

### Concatenating the two datasets side by side and save them to an other csv file

In [ ]:
# Concatenate the datasets by columns (side by side)
concatenated_df = pd.concat([superstore_df.reset_index(drop=True), 
                             retail_df.reset_index(drop=True)], axis=1)
concatenated_df.head()

In [ ]:
concatenated_df.columns

In [ ]:
#Saving the merged dataframe to a CSV file('concatenated_dataset.csv')
concatenated_df.to_csv('concatenated_dataset.csv', index=False)

In [ ]:
# Checking for missing values in both datasets
print(concatenated_df.isnull().sum())

The output indicates that the Superstore dataset has no missing values in its columns, while the Retail dataset has 8,994 missing values across multiple columns, , which is expected due to its smaller size, suggesting that additional data cleaning may be necessary for the Retail information.

In [ ]:
# Filling missing values for string columns (object type) with 'Unknown'
concatenated_df[concatenated_df.select_dtypes(include='object').columns] = concatenated_df.select_dtypes(include='object').fillna('Unknown')
# Filling missing values for numeric columns with 0
concatenated_df[concatenated_df.select_dtypes(include='number').columns] = concatenated_df.select_dtypes(include='number').fillna(0)
# Filling missing values for 'Retail_Order_Date' with a common date
concatenated_df['Retail_Order_Date'] = concatenated_df['Retail_Order_Date'].fillna(pd.Timestamp('2024-01-01')) 
concatenated_df.head()

In [ ]:
# Check for any duplicated in the merged dataframe and as you will see above there is no duplicated.
duplicates = concatenated_df.duplicated()
duplicates

### Data Exploration

In [ ]:
concatenated_df.describe()